# Answering Business Questions using SQL
In this project, we're going to practice using our SQL skills to answer business questions. Our starting point is a ficticious company called `Chinook` which sells music to customers. As an analyst, we might be given access to a database with tables associated with employees, customers, tracks/albums, transactions, and more. We will simulate this with the local database `chinook.db` which has the following schema diagram:

https://s3.amazonaws.com/dq-content/191/chinook-schema.svg

Let's start by connecting Jupyter to `chinook.db` so we can start exploring:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
# List of tables and views:

In [4]:
%%sql

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [10]:
# Simple list of artists

In [9]:
%%sql

SELECT *
  FROM artist
 LIMIT 5;

 * sqlite:///chinook.db
Done.


artist_id,name
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


## Selecting albums to purchase for Chinook 
Imagine that the Chinook store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name | Genre   |
|------|------|
|   Regal  | Hip-Hop |
|   Red Tone | Punk |
|   Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |

Further, suppose that this label represents artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [22]:
%%sql

WITH 
    usa_invoices AS
    (
        SELECT *
          FROM invoice
         INNER JOIN invoice_line AS inv_line ON inv_line.invoice_id = invoice.invoice_id
         WHERE billing_country = 'USA'
    ),
    track_and_genre AS
    (
        SELECT 
            track.track_id,
            genre.name AS genre
          FROM track
         INNER JOIN genre ON genre.genre_id = track.genre_id
    )
    
    
SELECT 
    tng.genre AS genre,
    SUM(us_inv.quantity) AS tracks_sold,
    CAST(SUM(us_inv.quantity) AS FLOAT) / CAST(
        (SELECT SUM(quantity) FROM usa_invoices) AS FLOAT) AS percentage_sold
  FROM usa_invoices AS us_inv
 INNER JOIN track_and_genre AS tng ON tng.track_id = us_inv.track_id
 GROUP BY tng.genre
 ORDER BY tracks_sold DESC
 LIMIT 10;

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


From the above results we might recommend purchasing the sale rights for the Punk group and advise against purchasing the rights for the Hip-Hop and Pop artists, with the Blues artist somewhere in the middle. We might also note that Rock is by far and away our most popular genre. On the other hand, we might also be suggest exploring the number of artists associated currently listed in the store with each genre in the interests of diversifying our offerings.

This just goes to show how the same result might yield very different actions depending on the priority!

## Analyzing Employee Sales Performance
Suppose each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

To that end, let's write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. Along the way, we'll add any extra attributes for that employee that might be relevant to the analysis.

In [38]:
%%sql

WITH
    sales_reps AS
    (
        SELECT *
          FROM employee
         WHERE title = "Sales Support Agent"
    ),
    
    sales_rep_totals AS
    (
        SELECT 
            cust.support_rep_id AS rep_id,
            SUM(inv.total) AS total_sales_assoc
          FROM customer AS cust
         INNER JOIN invoice AS inv ON inv.customer_id = cust.customer_id
         GROUP BY cust.support_rep_id
    )
    
SELECT 
    employee_id,
    last_name,
    first_name,
    title,
    hire_date,
    STRFTIME('%m', DATE('2017-12-31'))-STRFTIME('%m', hire_date) AS mo_since_hire,
    total_sales_assoc,
    total_sales_assoc/(STRFTIME('%m', DATE('2017-12-31'))-STRFTIME('%m', hire_date)) AS sales_per_mo_since_hire
  FROM sales_reps
 INNER JOIN sales_rep_totals ON sales_rep_totals.rep_id = sales_reps.employee_id
 ORDER BY total_sales_assoc DESC;

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,hire_date,mo_since_hire,total_sales_assoc,sales_per_mo_since_hire
3,Peacock,Jane,Sales Support Agent,2017-04-01 00:00:00,8,1731.510000000004,216.4387500000005
4,Park,Margaret,Sales Support Agent,2017-05-03 00:00:00,7,1584.0000000000032,226.28571428571473
5,Johnson,Steve,Sales Support Agent,2017-10-17 00:00:00,2,1393.9200000000028,696.9600000000014


Between the three sales agents, we notice a roughly 25% difference between the top representative (Jane Peacock) and the bottom (Steve Johnson). However, we also notice that this corresponds to the ordering of their hire dates. This suggests that the total sales associated with each representative is not enough to indicate their performace alone.

Instead, we might look at sales per month since a representatives hire date. Normally, we'd use `DATE('now')` to determine the number of months since a rep's hire date, but the data is from 2017 and this project is being completed in 2020. As a result, if we were to use `DATE('now')` to find the sales per month since hiring the difference between each employee would be minimized. Instead, for the purposes of this project we'll pretend that this report is being generated at the end of 2017 (i.e. on `2017-12-31`) and compute the sales per month relative to that date.

As a result, we find that although Jane has the greatest total of sales associated to her, Margaret's performance since hiring is roughly similar. On top of that, the representative with the fewest total sales associated, Steve, actually has the greatest number of sales per month since his hiring.

## Analyzing Sales by Country
Moving on, our next task is to  analyze the sales data for customers from each different country. Since both the `customer` and `invoice` tables have `country` columns, we will elect to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, you have been directed to calculate data, for each country, on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Additionally, any countries with only a single customer associated should be grouped as `other`.

In [66]:
%%sql

WITH 
    country_class AS
    (
        SELECT
            country,
            (
                CASE
                    WHEN COUNT(*) = 1 THEN 'Other'
                    ELSE country
                END
            ) AS country_class
          FROM customer
         GROUP BY country
    ),
    
    inv_with_country_class AS
    (
        SELECT 
            inv.invoice_id,
            inv.invoice_date,
            cc.country_class,
            cust.customer_id,
            inv.total
          FROM invoice AS inv
         INNER JOIN customer AS cust ON cust.customer_id = inv.customer_id
         INNER JOIN country_class AS cc ON cc.country = cust.country
    )
    
SELECT 
    country,
    n_customers,
    total_sales,
    sales_per_cust,
    average_order
  FROM
    (
        SELECT 
            country_class AS country,
            COUNT(DISTINCT(customer_id)) AS n_customers,
            SUM(total) AS total_sales,
            SUM(total) / COUNT(DISTINCT(customer_id)) AS sales_per_cust,
            SUM(total) / COUNT(DISTINCT(invoice_id)) AS average_order,
            CASE
                WHEN country_class = "Other" THEN 1
                ELSE 0
            END AS sort
          FROM inv_with_country_class 
         GROUP BY country        
    )
 ORDER BY sort ASC, total_sales DESC;


 * sqlite:///chinook.db
Done.


country,n_customers,total_sales,sales_per_cust,average_order
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,1094.9400000000003,72.99600000000002,7.44857142857143


## Purchasing decisions: albums or individual tracks?
Suppose that Chinook is set up in a way which allows users to purchase music in two ways:
1. purchase a whole album
2. purchase a collection of one or more individual tracks.

When a user makes a purchase it is recorded exclusively as either whole-album or individual tracks (i.e. a user cannot purchase an album and then add individual tracks in a single transaction). Finally customers are charged the same price whether they purchase albums as a whole or each track separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album. We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [84]:
%%sql

WITH
    iline_track_album AS
    (
        SELECT 
            invoice_id,
            invoice_line.track_id,
            track.album_id
          FROM invoice_line 
         INNER JOIN track ON track.track_id = invoice_line.track_id
         GROUP BY invoice_id
    )

SELECT 
    album_purchase,
    COUNT(*) AS n_invoice,
    CAST(COUNT(*) AS FLOAT)/(SELECT COUNT(*) FROM invoice) AS percent_purchases
  FROM
        (
        SELECT
            itla.*,
            CASE
                WHEN 
                    (
                        SELECT inv_line.track_id FROM invoice_line AS inv_line
                         WHERE inv_line.invoice_id = itla.invoice_id


                        EXCEPT


                        SELECT track.track_id FROM track
                         WHERE track.album_id = itla.album_id 
                    ) IS NULL
                    AND
                    (
                        SELECT track.track_id FROM track
                         WHERE track.album_id = itla.album_id 

                        EXCEPT

                        SELECT inv_line.track_id FROM invoice_line AS inv_line
                         WHERE inv_line.invoice_id = itla.invoice_id
                    ) IS NULL
                    THEN 1
                    ELSE 0
                END AS album_purchase
          FROM iline_track_album AS itla
        )
 GROUP BY album_purchase

 * sqlite:///chinook.db
Done.


album_purchase,n_invoice,percent_purchases
0,500,0.8143322475570033
1,114,0.18566775244299674
